In [1]:
# IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS
import os
import cv2
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, auc, roc_curve, roc_auc_score, recall_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers     import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalMaxPool2D
from tensorflow.keras.models     import Sequential
from tensorflow.keras.callbacks  import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses     import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop

from random import shuffle
from tqdm   import tqdm # permite verificar o progresso do carregamento


class PrepareData:

  def __init__(self):
    path = '/content/drive/My Drive/Colab Notebooks/imageUNMASKER/'
    self.train_path = path + 'dataset-proj/train/'  
    self.validation_path = path + 'dataset-proj/validation/' 
    self.outputs_path = path + 'outputs/presentation/'
    self.resize = (300, 300)
    self.early_callback = EarlyStopping(monitor = 'loss', patience = 10)

  def label_img(self, _path):
    word_label = _path.split('/')[-2]
    return [1,0] if word_label == 'fake' else [0,1]

  def scale_image(self, img):
    scale = self.resize
    img = cv2.resize(img, scale)
    return img

  def load_prepare_data (self, _type):
    dataset = []
    _path = self.train_path if _type == 'train' else self.validation_path 
    
    for folder in os.listdir(_path): #['fake', 'real']
      path_folder = os.path.join(_path, folder)
      
      for filename in tqdm(os.listdir(path_folder)):
        path = os.path.join(path_folder, filename)
        label = self.label_img(path) # [0,1] para real , [1,0] para fake

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = self.scale_image(img)
        img = img.astype("float64") / 255.0 
        
        dataset.append([np.array(img), np.array(label)])
    
    shuffle(dataset)
    return dataset    

  def model_train_and_evaluate(self, result_table, model, trainX, trainY, validationX, validationY, n_epochs = 50, key=""):
    
    early_callback = self.early_callback 
    checkpoint  = ModelCheckpoint(self.outputs_path + 'saved_model.h5', 
                                  monitor='val_accuracy', 
                                  mode='max', 
                                  verbose=1, 
                                  save_best_only=True)
    
    X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.20, random_state=42)

    history = model.fit(X_train,
                        y_train,
                        epochs = n_epochs,
                        validation_data = (X_test, y_test),
                        verbose = 1
                        #,callbacks = [early_callback, checkpoint] 
                        )
    
    n_epochs = len(history.history['loss'])

    visualize_fit_summary(history, n_epochs)

    ypredicted = model.predict(validationX)
    
    yval  = validationY.argmax(axis=1)
    ypred = ypredicted.argmax(axis=1)

    fpr, tpr, _ = roc_curve(yval, ypred)

    auc         = roc_auc_score(yval, ypred)
    rscore      = recall_score(yval, ypred)
    acc         = accuracy_score(yval, ypred)
    conf_m      = confusion_matrix(yval, ypred)
    result_table = result_table.append({'classifiers':model.__class__.__name__+"-"+key,
                                        'auc': auc, 
                                        'recall':rscore,
                                        'acc':acc,
                                        'confusion_matrix': conf_m}, ignore_index=True)
    
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='red', 
            lw=lw, label='ROC curve (area = %0.2f)' % auc)
    plt.plot([0, 1], [0, 1], 
            color='darkblue', lw=lw, 
            linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

    return result_table


def visualize_fit_summary(hist, n_epochs):
  acc = hist.history['accuracy']
  val_acc = hist.history['val_accuracy']

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']

  epochs_range = range(n_epochs)

  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()


pData = PrepareData()

In [ ]:
train_data = pData.load_prepare_data(_type = 'train')
print("train_data Shape: ", np.asarray(train_data).shape)

100%|██████████| 700/700 [08:20<00:00,  1.40it/s]

train_data Shape:  (1400, 2)



/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
validation_data = pData.load_prepare_data(_type = 'validation')
print("validation_data Shape: ", np.asarray(validation_data).shape)

In [ ]:
train_images = np.array([i[0] for i in train_data]).reshape(-1, 300, 300, 1)
train_labels = np.array([i[1] for i in train_data]).astype('float64')

print("Train Images, shape and type: ", train_images.shape, train_images.dtype)
print("Train Labels, shape and type: ", train_labels.shape, train_labels.dtype)

validation_images = np.array([i[0] for i in validation_data]).reshape(-1, 300, 300, 1)
validation_labels = np.array([i[1] for i in validation_data]).astype('float64')

print("Validation Images, shape and type: ", validation_images.shape, validation_images.dtype)
print("Validation Labels, shape and type: ", validation_labels.shape, validation_labels.dtype)

In [ ]:
# train_images[1]

In [ ]:
result_table = pd.DataFrame(columns=['classifiers',
                                     'auc', 
                                     'recall', 
                                     'acc', 
                                     'confusion_matrix'])
print(result_table)

inputs_shape = validation_images[0].shape
print("Inputs Shape: ", inputs_shape)

Empty DataFrame
Columns: [classifiers, auc, recall, acc, confusion_matrix]
Index: []
Inputs Shape:  (300, 300, 1)


In [ ]:
model = Sequential([
    Conv2D(64,3, activation='relu', input_shape=inputs_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3)),

    Conv2D(128,2, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128,2, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.2),
    
    Conv2D(256,2, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(256,2, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Dropout(0.2),
    
    Conv2D(256,2, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(300, activation='relu'),
    Dense(300, activation='relu'),
    Dense(2,activation='softmax') 
])

lr = 0.001
n_epochs = 100

opt = Adam(learning_rate = lr)

model.summary()
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 298, 298, 64)      640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 298, 298, 64)      256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 98, 98, 128)       32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 98, 98, 128)       512       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 48, 128)      

In [ ]:
result_table = pData.model_train_and_evaluate(result_table,
                                              model, 
                                              train_images,
                                              train_labels,
                                              validation_images,
                                              validation_labels,
                                              n_epochs = n_epochs,
                                              key = opt.__class__.__name__ + ' ' + str(n_epochs) + 'ep. lr' + str(lr)
                                              )

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(result_table)

                      classifiers    auc    recall    acc  \
0  Sequential-Adam 100ep. lr0.001  0.775  0.726667  0.775   

         confusion_matrix  
0  [[247, 53], [82, 218]]  


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 256, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(BatchNormalization())
  
  model.add(Conv2D(64, 2, activation='relu'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Dropout(0.1))
  
  model.add(Conv2D(128, 2, activation='relu'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  # compile model
  opt = Adam()
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  return model

model2 = define_model()

train_datagen = ImageDataGenerator (rescale = 1.0/255.0, 
                                    horizontal_flip = True
                                   #,brightness_range= [0.2, 1]
                                    )

validation_datagen = ImageDataGenerator (rescale = 1.0/255.0)

train_it = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/tdx_project/dataset-proj/train/',
	class_mode='binary', batch_size=64, target_size=(256, 256))
test_it = validation_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/tdx_project/dataset-proj/validation/',
	class_mode='binary', batch_size=64, target_size=(256, 256))

history = model2.fit(train_it, 
                     steps_per_epoch=len(train_it), 
                     validation_data=test_it, 
                     validation_steps=len(test_it), 
                     #callbacks = [early_callback],
                     epochs=50)


Found 1400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Epoch 1/50
22/22 [==============================] - 1129s 51s/step - loss: 2.3101 - accuracy: 0.4914 - val_loss: 0.6960 - val_accuracy: 0.5000
Epoch 2/50
22/22 [==============================] - 79s 4s/step - loss: 0.6747 - accuracy: 0.5879 - val_loss: 0.7381 - val_accuracy: 0.5017
Epoch 3/50
22/22 [==============================] - 78s 4s/step - loss: 0.5892 - accuracy: 0.7000 - val_loss: 0.7986 - val_accuracy: 0.5083
Epoch 4/50
22/22 [==============================] - 78s 4s/step - loss: 0.4976 - accuracy: 0.7671 - val_loss: 0.6329 - val_accuracy: 0.6133
Epoch 5/50
22/22 [==============================] - 78s 4s/step - loss: 0.4444 - accuracy: 0.8150 - val_loss: 0.6047 - val_accuracy: 0.6800
Epoch 6/50
22/22 [==============================] - 80s 4s/step - loss: 0.3243 - accuracy: 0.8607 - val_loss: 0.6457 - val_accuracy: 0.6533
Epoch 7/50
22/22 [==============================] - 80s 4s/step - loss: 0.

In [ ]:
history = model2.fit(train_it, 
                     steps_per_epoch=len(train_it), 
                     validation_data=test_it, 
                     validation_steps=len(test_it), 
                     #callbacks = [early_callback],
                     epochs=100)


Epoch 1/100
22/22 [==============================] - 82s 4s/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 1.1568 - val_accuracy: 0.8067
Epoch 2/100
22/22 [==============================] - 79s 4s/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 1.4019 - val_accuracy: 0.7683
Epoch 3/100
22/22 [==============================] - 81s 4s/step - loss: 0.0048 - accuracy: 0.9993 - val_loss: 1.3339 - val_accuracy: 0.8067
Epoch 4/100
22/22 [==============================] - 80s 4s/step - loss: 0.0184 - accuracy: 0.9957 - val_loss: 1.2983 - val_accuracy: 0.8017
Epoch 5/100
22/22 [==============================] - 82s 4s/step - loss: 0.0148 - accuracy: 0.9936 - val_loss: 1.2093 - val_accuracy: 0.7267
Epoch 6/100
22/22 [==============================] - 81s 4s/step - loss: 0.0255 - accuracy: 0.9907 - val_loss: 1.0056 - val_accuracy: 0.8100
Epoch 7/100
22/22 [==============================] - 83s 4s/step - loss: 0.0262 - accuracy: 0.9914 - val_loss: 1.7042 - val_accuracy: 0.6817
Epoch 8/100
2